# 사용자 기반 미로게임 만들기

In [38]:
# 플레이어 이미지 사이즈 변경
img = Image.open('./ghost.png')
img2 = img.resize((40, 40))
img2.save('./ghost2.png')

In [66]:
import tkinter
from PIL import ImageTk, Image

keyInput = '' # 키 입력 전역변수 선언

def key_Down(i) : # 키를 입력할 때
    global keyInput
    keyInput = i.keysym
    print(keyInput)
    
def key_Up(i) : # 키를 입력하고 뗐을 때
    global keyInput
    keyInput = '' # 키 값 다시 초기화
    
# postion 초기값
posX, posY = 1, 1

def playerMove() : # 키 입력 시 실시간 진행
    global posX, posY
    # 
    if keyInput == 'Up' and gameMap[posY-1][posX] == 0 :
        posY = posY - 1
    if keyInput == 'Down' and gameMap[posY+1][posX] == 0 :
        posY = posY + 1
    if keyInput == 'Right' and gameMap[posY][posX+1] == 0 :
        posX = posX + 1
    if keyInput == 'Left' and gameMap[posY][posX-1] == 0 :
        posX = posX - 1
        
    canvas.coords('player', posX * 40 + 20, posY * 40 + 20) # 플레이어 이동
    tk.after(100, playerMove) # 0.1초 후에 실행

tk = tkinter.Tk()
tk.title('미로 찾기')
tk.bind('<KeyPress>', key_Down)
tk.bind('<KeyRelease>', key_Up)
canvas = tkinter.Canvas(width = 1000, height = 240, bg = 'white')
# canvas.create_rectangle(15, 15, 200, 200, fill = 'red', outline = 'green', width = 10)
canvas.pack()
gameMap = [
    [1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1], 
    [1,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1], 
    [1,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1], 
    [1,0,1,0,0,0,0,1,0,0,0,1,0,0,1,0,0,1,0,0,0,0,1,0,1], 
    [1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1], 
    [1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1]
]
for y in range(6) :
    for x in range(25) :
        # 해당 좌표의 값이 1인 경우 색칠하기 (벽 만들기)
        if gameMap[y][x] == 1 :
            canvas.create_rectangle(x * 40, y * 40, x * 40 + 40, y * 40 + 40, fill = 'green', outline = 'black', width = 5)

ghost = tkinter.PhotoImage(file = './ghost2.png')
canvas.create_image(posX * 40 + 20, posY * 40 + 20, image = ghost, tag = 'player') # 플레이어 배치
playerMove() # 플레이어 이동 함수
tk.mainloop()

# 몬테카를로

In [ ]:
import time
import numpy as np
import tkinter as tk
from PIL import ImageTk, Image

np.random.seed(1)
PhotoImage = ImageTk.PhotoImage
UNIT = 100  # 픽셀 수
HEIGHT = 5  # 그리드 월드 가로
WIDTH = 5  # 그리드 월드 세로


class Env(tk.Tk):
    def __init__(self):
        super(Env, self).__init__()
        self.action_space = ['u', 'd', 'l', 'r']
        self.n_actions = len(self.action_space)
        self.title('SARSA')
        self.geometry('{0}x{1}'.format(HEIGHT * UNIT, HEIGHT * UNIT))
        self.shapes = self.load_images()
        self.canvas = self._build_canvas()
        self.texts = []

    def _build_canvas(self):
        canvas = tk.Canvas(self, bg='white',
                           height=HEIGHT * UNIT,
                           width=WIDTH * UNIT)
        # 그리드 생성
        for c in range(0, WIDTH * UNIT, UNIT):  # 0~400 by 80
            x0, y0, x1, y1 = c, 0, c, HEIGHT * UNIT
            canvas.create_line(x0, y0, x1, y1)
        for r in range(0, HEIGHT * UNIT, UNIT):  # 0~400 by 80
            x0, y0, x1, y1 = 0, r, HEIGHT * UNIT, r
            canvas.create_line(x0, y0, x1, y1)

        # 캔버스에 이미지 추가
        self.rectangle = canvas.create_image(50, 50, image=self.shapes[0])
        self.triangle1 = canvas.create_image(250, 150, image=self.shapes[1])
        self.triangle2 = canvas.create_image(150, 250, image=self.shapes[1])
        self.circle = canvas.create_image(250, 250, image=self.shapes[2])

        canvas.pack()

        return canvas

    def load_images(self):
        rectangle = PhotoImage(
            Image.open("./img/rectangle.png").resize((65, 65)))
        triangle = PhotoImage(
            Image.open("./img/triangle.png").resize((65, 65)))
        circle = PhotoImage(
            Image.open("./img/circle.png").resize((65, 65)))

        return rectangle, triangle, circle

    def text_value(self, row, col, contents, action, font='Helvetica', size=10,
                   style='normal', anchor="nw"):
        if action == 0:
            origin_x, origin_y = 7, 42
        elif action == 1:
            origin_x, origin_y = 85, 42
        elif action == 2:
            origin_x, origin_y = 42, 5
        else:
            origin_x, origin_y = 42, 77

        x, y = origin_y + (UNIT * col), origin_x + (UNIT * row)
        font = (font, str(size), style)
        text = self.canvas.create_text(x, y, fill="black", text=contents,
                                       font=font, anchor=anchor)
        return self.texts.append(text)

    def print_value_all(self, q_table):
        for i in self.texts:
            self.canvas.delete(i)
        self.texts.clear()
        for x in range(HEIGHT):
            for y in range(WIDTH):
                for action in range(0, 4):
                    state = [x, y]
                    if str(state) in q_table.keys():
                        temp = q_table[str(state)][action]
                        self.text_value(y, x, round(temp, 3), action)

    def coords_to_state(self, coords):
        x = int((coords[0] - 50) / 100)
        y = int((coords[1] - 50) / 100)
        return [x, y]

    def reset(self):
        self.update()
        time.sleep(0.5)
        x, y = self.canvas.coords(self.rectangle)
        self.canvas.move(self.rectangle, UNIT / 2 - x, UNIT / 2 - y)
        self.render()
        return self.coords_to_state(self.canvas.coords(self.rectangle))

    def step(self, action):
        state = self.canvas.coords(self.rectangle)
        base_action = np.array([0, 0])
        self.render()

        if action == 0:  # 상
            if state[1] > UNIT:
                base_action[1] -= UNIT
        elif action == 1:  # 하
            if state[1] < (HEIGHT - 1) * UNIT:
                base_action[1] += UNIT
        elif action == 2:  # 좌
            if state[0] > UNIT:
                base_action[0] -= UNIT
        elif action == 3:  # 우
            if state[0] < (WIDTH - 1) * UNIT:
                base_action[0] += UNIT

        # 에이전트 이동
        self.canvas.move(self.rectangle, base_action[0], base_action[1])
        # 에이전트(빨간 네모)를 가장 상위로 배치
        self.canvas.tag_raise(self.rectangle)
        next_state = self.canvas.coords(self.rectangle)

        # 보상 함수
        if next_state == self.canvas.coords(self.circle):
            reward = 100
            done = True
        elif next_state in [self.canvas.coords(self.triangle1),
                            self.canvas.coords(self.triangle2)]:
            reward = -100
            done = True
        else:
            reward = 0
            done = False

        next_state = self.coords_to_state(next_state)
        return next_state, reward, done

    def render(self):
        time.sleep(0.03)
        self.update()

In [67]:
class Agent() :
    def __init__(self) :
        pass
    
    def select_actions(self) :
        coin = random.random()
        if coin < 0.25 :
            action = 0
        elif coin < 0.5 :
            action = 1
        elif coin < 0.75 :
            action = 2
        else :
            action = 3
        return action

In [ ]:
def main() :
    env = Env()
    agent = Agent()
    data = [[0,0,0,0], [0,0,0,0], [0,0,0,0], [0,0,0,0]] # 테이블 초기화
    gamma = 1.0
    alpha = 0.0001
    
    for k in range(50000) : # 총 5만 번의 에피소드 진행
        done = False
        history = []
        while not done :
            action = agent.select_action()
            (x,y), reward, done = env.step(action)
            history.append((x,y,reward))
        env.reset()
        
        # 매 에피소드가 끝나고 바로 해당 데이터를 이용해 테이블을 업데이트
        cum_reward = 0
        for transition in history[::-1] :
            # 방문했던 상태들을 뒤에서부터 보며 차례차례 리턴을 계산
            x, y, reward = transition
            data[x][y] = data[x][y] + alpha*(cum_reward - data[x][y])
            cum_reward = reward + gamma * cum_reward
            
        # 학습이 끝나고 난 후 데이터를 출력해보기 위한 코드
        for row in data :
            print(row)